In [1]:
# %% [markdown]
# # NFL Play Predictor - Interactive Dashboard
# Live inference tool using the pre-trained SGD model.

# %%
import sys
import os
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from nfl_utils import TEAM_MAPPING, PLAY_TYPE_MAPPING

# Load Model (XGBoost)
MODEL_PATH = '../data/models/nfl_xgb_pipeline.joblib'

if not os.path.exists(MODEL_PATH):
    print("Model not found! Please run '01_model_training.ipynb' first.")
else:
    model = joblib.load(MODEL_PATH)
    print("Model loaded ready for inference.")


Model loaded ready for inference.


In [ ]:
# %% [markdown]
# ## Prediction Interface

# %%
# --- UI Configuration ---
style = {'description_width': 'initial'}
layout_half = widgets.Layout(width='45%')

# Inputs
w_year = widgets.IntText(value=2024, description='Season:', style=style, layout=layout_half)
w_qtr = widgets.Dropdown(options=[1, 2, 3, 4, 5], value=1, description='Quarter:', style=style, layout=layout_half)

w_min = widgets.IntSlider(value=15, min=0, max=15, description='Minutes Left:', style=style)
w_sec = widgets.IntSlider(value=0, min=0, max=59, description='Seconds Left:', style=style)

w_down = widgets.Dropdown(options=[1, 2, 3, 4], value=1, description='Down:', style=style, layout=layout_half)
w_togo = widgets.IntText(value=10, description='Yards To Go:', style=style, layout=layout_half)
w_yardline = widgets.IntSlider(value=25, min=1, max=99, description='Yard Line (1-99):', style=style)

# Team Selectors (Sorted for usability)
unique_teams = sorted(list(set(TEAM_MAPPING.keys())))
w_off = widgets.Dropdown(options=unique_teams, value='KC', description='Offense:', style=style, layout=layout_half)
w_def = widgets.Dropdown(options=unique_teams, value='SF', description='Defense:', style=style, layout=layout_half)

btn_predict = widgets.Button(description="Predict Next Play", button_style='primary', icon='football-ball')
out_display = widgets.Output()

In [3]:
# --- Logic ---
def on_predict(_):
    with out_display:
        clear_output()
        
        # Encode Teams
        off_id = TEAM_MAPPING[w_off.value]
        def_id = TEAM_MAPPING[w_def.value]
        
        # Construct Feature Vector
        # [SeasonYear, Quarter, Minute, Second, Down, ToGo, YardLine, OFF, DEF]
        features = [[
            w_year.value, w_qtr.value, w_min.value, w_sec.value,
            w_down.value, w_togo.value, w_yardline.value,
            off_id, def_id
        ]]
        
        try:
            pred_class = model.predict(features)[0]
            pred_label = PLAY_TYPE_MAPPING.get(pred_class, "Unknown")
            prob = model.predict_proba(features).max()
            
            # Rich Output
            print(f"🏈 Prediction: {pred_label.upper()}")
            print(f"Confidence: {prob:.1%}")
            print(f"Scenario: {w_off.value} vs {w_def.value} | Down: {w_down.value} & {w_togo.value}")
            
        except Exception as e:
            print(f"Error: {e}")

btn_predict.on_click(on_predict)

In [4]:
# --- Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>Game Situation</h3>"),
    widgets.HBox([w_year, w_qtr]),
    widgets.HBox([w_min, w_sec]),
    widgets.HTML("<hr>"),
    widgets.HBox([w_down, w_togo]),
    w_yardline,
    widgets.HTML("<hr>"),
    widgets.HBox([w_off, w_def]),
    widgets.HTML("<br>"),
    btn_predict,
    out_display
])

display(ui)